# Recreating Fig. 2

The paper [NOISE OR SIGNAL: THE ROLE OF IMAGE BACKGROUNDS IN OBJECT RECOGNITION](https://openreview.net/pdf?id=gl3D-xY7wLq) explores the effect of background on predictive deep learning models.


![title](Capture.png)

In this paper they show the following accuracies can be achieved on their models when training them on modified data then testing them on the original data. 

Our goal is to recreate these results

# 1. Setting up the data

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from collections import Counter
import torchvision.models as models
import torchvision.transforms.functional as TF
import os
import matplotlib.pyplot as plt 
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Southampton/Year\ 4/Semester\ 2/Git/GroupWork/Deep\ Learning/

/content/drive/MyDrive/Southampton/Year 4/Semester 2/Git/GroupWork/Deep Learning


In [ ]:
ls data/

no_fg/     no_fg.tar.gz      only_bg_t.tar.gz  original.tar.gz
no_fg.rar  only_bg_b.tar.gz  original/


In [ ]:
!tar -xvf data/only_bg_b.tar.gz

Streaming output truncated to the last 5000 lines.
only_bg_b/train/00_dog/n02105641_1411.JPEG
only_bg_b/train/00_dog/n02098105_195.JPEG
only_bg_b/train/00_dog/n02105412_7465.JPEG
only_bg_b/train/00_dog/n02088466_8904.JPEG
only_bg_b/train/00_dog/n02091467_1213.JPEG
only_bg_b/train/00_dog/n02086079_13106.JPEG
only_bg_b/train/00_dog/n02088632_1583.JPEG
only_bg_b/train/00_dog/n02090379_291.JPEG
only_bg_b/train/00_dog/n02093991_1310.JPEG
only_bg_b/train/00_dog/n02086240_3681.JPEG
only_bg_b/train/00_dog/n02095570_4161.JPEG
only_bg_b/train/00_dog/n02098105_2460.JPEG
only_bg_b/train/00_dog/n02086240_4711.JPEG
only_bg_b/train/00_dog/n02113186_12758.JPEG
only_bg_b/train/00_dog/n02096294_484.JPEG
only_bg_b/train/00_dog/n02090721_3527.JPEG
only_bg_b/train/00_dog/n02099849_2315.JPEG
only_bg_b/train/00_dog/n02091635_386.JPEG
only_bg_b/train/00_dog/n02086646_1721.JPEG
only_bg_b/train/00_dog/n02090622_8516.JPEG
only_bg_b/train/00_dog/n02090622_2931.JPEG
only_bg_b/train/00_dog/n02105412_3065.JPEG
only_

In [ ]:
!tar -xvf data/original.tar.gz

Streaming output truncated to the last 5000 lines.
original/train/00_dog/n02105641_1411.JPEG
original/train/00_dog/n02098105_195.JPEG
original/train/00_dog/n02105412_7465.JPEG
original/train/00_dog/n02088466_8904.JPEG
original/train/00_dog/n02091467_1213.JPEG
original/train/00_dog/n02086079_13106.JPEG
original/train/00_dog/n02088632_1583.JPEG
original/train/00_dog/n02090379_291.JPEG
original/train/00_dog/n02093991_1310.JPEG
original/train/00_dog/n02086240_3681.JPEG
original/train/00_dog/n02095570_4161.JPEG
original/train/00_dog/n02098105_2460.JPEG
original/train/00_dog/n02086240_4711.JPEG
original/train/00_dog/n02113186_12758.JPEG
original/train/00_dog/n02096294_484.JPEG
original/train/00_dog/n02090721_3527.JPEG
original/train/00_dog/n02099849_2315.JPEG
original/train/00_dog/n02091635_386.JPEG
original/train/00_dog/n02086646_1721.JPEG
original/train/00_dog/n02090622_8516.JPEG
original/train/00_dog/n02090622_2931.JPEG
original/train/00_dog/n02105412_3065.JPEG
original/train/00_dog/n0210

In [ ]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])
train = datasets.ImageFolder('only_bg_t/train', transform=transform)
test = datasets.ImageFolder('only_bg_t/val', transform=transform)

In [ ]:
N = {"train" : len(train), "test" : len(test)} 

In [ ]:
names = ["dog", "bird", "wheeled_vehicle", "reptile", "carnivore", "insect", "musical_instrument", "primate", "fish"]

In [ ]:
dic = {}
for i in range(len(names)):
    dic[i] = names[i]
c = dict(Counter(train.targets))



In [ ]:
for i in range(len(c)):
    c[dic[i]] = c.pop(i)

In [ ]:
batchsize = 256
dataloaders = {"train" : torch.utils.data.DataLoader(train, batch_size=batchsize, shuffle=True),
               "test" :  torch.utils.data.DataLoader(test,  batch_size=batchsize, shuffle=True) } 

In [ ]:
images, labels = next(iter(dataloaders["train"]))
im = np.asarray(images[0])
im = np.moveaxis(im, 0, -1)
print(im.shape)
print(dic[int(labels[0])])
plt.imshow(im)

# 2. Transfer Learning

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from google.colab import files

In [ ]:
device

In [ ]:
model = models.resnet18(pretrained = False).to(device)

In [ ]:
model.fc = nn.Sequential(
               nn.Linear(model.fc.in_features, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 9)).to(device)

In [ ]:
model.load_state_dict(torch.load("only_bg_b_resnet18_10_epochs_sgd.weights"))

In [ ]:
for param in model.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### 2.1 Training

In [ ]:
! pip install torchbearer

In [ ]:
import gc
gc.collect()

torch.cuda.empty_cache()

In [ ]:
import torchbearer
from torchbearer import Trial
from torch import optim

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimizer, criterion, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(dataloaders["train"], test_generator=dataloaders["test"])
trial.run(epochs=10)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print()
print(results)
torch.save(model.state_dict(), "only_bg_b_resnet18_20_epochs_sgd.weights")

from google.colab import files
files.download('only_bg_b_resnet18_20_epochs_sgd.weights')

### 2.2 Testing on Original

In [ ]:
import torchbearer
from torchbearer import Trial
from torch import optim

In [ ]:
original = datasets.ImageFolder('original/val', transform=transform)

In [ ]:
original_loader =  torch.utils.data.DataLoader(original, batch_size=batchsize, shuffle=True)

In [ ]:
N_or = len(original)

In [ ]:
N_or

In [ ]:
images, labels = next(iter(original_loader))
im = np.asarray(images[0])
im = np.moveaxis(im, 0, -1)
print(im.shape)
print(dic[int(labels[0])])
plt.imshow(im)

In [ ]:
trial = Trial(model, optimizer, criterion, metrics=['loss', 'accuracy']).to(device)
trial.with_test_generator(original_loader)
trial.run(1)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)

In [ ]:
results

## 3. My own model

In [ ]:
from torchbearer import Trial
import torch.nn.functional as F

In [ ]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, (31, 31), padding=0)
        self.conv2 = nn.Conv2d(10, 10, (16, 16), padding=0)
        self.conv3 = nn.Conv2d(10, 5, (3, 3), padding=2)
        self.fc1 = nn.Linear(2205, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 9)
    
    def forward(self, x):
        # YOUR CODE HERE
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        
        out = self.conv2(out)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        
        
        out = self.conv3(out)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        
        out = F.dropout(out, 0.2)
        
        out = out.view(out.shape[0], -1)
        
        
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        
        return out

In [ ]:
model = CNN()
print(sum([i.numel() for i in model.parameters()]))

5626498


In [ ]:
#reset the data loaders
torch.manual_seed(seed)
# build the model
model = CNN()

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(dataloaders["train"], test_generator=dataloaders["test"])
trial.run(epochs=10)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

#assert results['test_acc'] > 0.99


0/10(t):   0%|          | 0/710 [00:00<?, ?it/s]